In [1]:
from nltk.tokenize.casual import casual_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re
import pandas as pd
import string
import os
base = "../../data/clean/"
pd.set_option('display.max_rows', 150)

In [2]:
tweetdf = pd.read_csv(base+'trisma2016_cleaned_text_by_lga_and_time_of_day.csv')
tweetdf = tweetdf.sort_values(by='lga')
tweetdf.head()

,Unnamed: 0,lga,time_of_day,text,cleaned
0,0,Albury,afternoon,"I can't say if its ""personally directed"" @ me ...",can't say if its personally directed me but wh...
1,1,Albury,evening,New year resolution \n\nFind another job so yo...,new year resolution find another job so you do...
2,2,Albury,morning,@BluesTumut @Group9RL going to be a smashing s...,going to be smashing season this year good luc...
3,3,Albury,night,#2015bestnine🌟 #BestOfNine2015 #byebye #2015 #...,#2015bestnine #bestofnine2015 #byebye #2015 #h...
4,4,Armidale Regional,afternoon,#تنفيذ_القصاص_على_47_ارهابي\n\nليسوا على مذهب ...,#tnfydh_lqss_ _47_rhby lysw mdhhb whd wl jnsy ...


In [3]:
tweetdf = tweetdf.fillna(0)

In [4]:
import shapefile
from shapely.geometry import shape, Point, mapping
import numpy as np
import pandas as pd

raw = "../../data/raw/"
clean = "../../data/processed/"
filename = "2016_LGA_SHAPE/LGA_2016_AUST"

sf = shapefile.Reader(raw+filename)

nswshapes = list(map(lambda x: shape(x.__geo_interface__), sf.shapes()[:130]))
centroids = list(map(lambda x: shape(x.__geo_interface__).centroid, sf.shapes()[:130]))
df = pd.DataFrame(sf.records())
nswdf = df[df[4] =="New South Wales"]
# drop "No usual address" - not a useful LGA
nswdf = nswdf[nswdf[0] != "LGA19499"]
nswdf.columns = ["code", "codenum", "name", "unknown", "state", "unknown2"]
nswdf["clean_name"] = nswdf["name"].str.extract('([^\\(]*)', expand=False).str.strip()
nswdf

,code,codenum,name,unknown,state,unknown2,clean_name
0,LGA10050,10050,Albury (C),1,New South Wales,305.9459,Albury
1,LGA10130,10130,Armidale Regional (A),1,New South Wales,8620.6990,Armidale Regional
2,LGA10250,10250,Ballina (A),1,New South Wales,484.9389,Ballina
3,LGA10300,10300,Balranald (A),1,New South Wales,21690.6753,Balranald
4,LGA10470,10470,Bathurst Regional (A),1,New South Wales,3817.8646,Bathurst Regional
5,LGA10550,10550,Bega Valley (A),1,New South Wales,6278.8811,Bega Valley
6,LGA10600,10600,Bellingen (A),1,New South Wales,1600.4337,Bellingen
7,LGA10650,10650,Berrigan (A),1,New South Wales,2065.7759,Berrigan
8,LGA10750,10750,Blacktown (C),1,New South Wales,240.0487,Blacktown
9,LGA10800,10800,Bland (A),1,New South Wales,8557.6535,Bland


In [5]:
#amalgamations on shpfile
amalgamations = {"Botany Bay":"Bayside",
                 "Rockdale":"Bayside",
                 "Greater Hume Shire": "Greater Hume",
                 "Gundagai": 'Cootamundra-Gundagai',
                 'Western Plains Regional': "Dubbo Regional",
                 'Unincorporated NSW': 'Unincorporated Far West',
                 
                }

for x in amalgamations.keys():
    nswdf.loc[nswdf["clean_name"] == x, "clean_name"] =  amalgamations[x]

In [6]:
import json
lga_json = list(map(lambda x: x.__geo_interface__, sf.shapes()[:130]))

geojson = {'type': 'FeatureCollection',
            'features': [{'type':'Feature', 'geometry':x, 'properties': {'name':nswdf.iloc[i]['clean_name']}} for i, x in enumerate(lga_json)]
          }

#with open('geojson.json','w') as f:
#    json.dump(geojson, f)

In [19]:
aggdfs = []
tokens = ["tree", "lightning", "#qlder", "fun", "goat"]
for token in tokens:
    tweetdf.loc[:,token] = tweetdf["cleaned"].str.count("[$\W]"+token+"[\W^]")
    aggdfs.append(tweetdf.groupby("lga").sum())

In [20]:
aggdfs[-3]

,Unnamed: 0,tree,lightning,#qlder,fun,goat
lga,,,,,,
Albury,6,18,2,0,395,13
Armidale Regional,22,14,0,0,82,2
Ballina,38,14,1,0,71,7
Balranald,54,1,0,0,5,0
Bathurst Regional,70,12,0,0,79,0
Bayside,86,3,2,0,51,0
Bega Valley,102,15,2,0,62,0
Bellingen,118,7,0,0,24,4
Berrigan,134,2,0,0,7,0


In [17]:
def colorshp(col, cmax, cmin):
    portion = (col-cmin)
    if portion > 0:
        portion = np.log(portion)
    point = portion/np.log((cmax - cmin))
    #print(point)
    strength = int(point * 255)
    alpha = point
    return 'rgba({0},0,0,{1:.2f})'.format(strength,alpha)

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
from plotly.graph_objs import *

app = dash.Dash()
os.environ['WERKZEUG_RUN_MAIN'] = 'true'

data = []
for i, tok in enumerate(tokens):
    #lga boundaries
    aggdf = aggdfs[i]
    lga_layers = list(map(lambda x: { 
                    'sourcetype' : "geojson",
                    'source': x,
                    'type'   : 'fill',
                    'color': colorshp(aggdf.loc[x["properties"]["name"]][tok], aggdf[tok].max(),0),
                    }, geojson['features']))

    #lga labels
    labels = dict(type='scattermapbox',
                lon=[x.x for x in centroids],
                lat=[x.y for x in centroids],
                mode='text',
                text=list(nswdf.clean_name),
                hoverinfo = 'none',
                 )
    #counts for hover
    counts = dict(type='scattermapbox',
                lon=[x.x for x in centroids],
                lat=[x.y for x in centroids],
                mode='none',
                text = aggdf[tok].loc[list(nswdf.clean_name)],
                hoverinfo = 'text',
                 )
    data.append({
            "data": [labels, counts],
            "layout": go.Layout(
                hovermode = "closest",
                margin = dict(l = 0, r = 0, t = 0, b = 0),
                mapbox = dict(
                    layers = lga_layers,
                    accesstoken = "pk.eyJ1IjoiZGVzdWx0aXIiLCJhIjoiY2pmZDN4M2FlMTEzYjJ3bWlyc2VhaDkxaiJ9.VvezwDLTQlh2GVAEsauwlg",
                    bearing = 0,
                    center = dict(lat=-32.662833, lon=147.031917),
                    style = "light",
                    pitch = 0,
                    zoom = 5.5,
                )
            )
        })
    
app.layout = html.Div(children=[
    html.H4(children='Tokens by LGA'),
    dcc.Slider(
        id='slider',
        min=0,
        max=len(tokens),
        step=1,
        value=0,
        marks={i:tokens[i] for i in range(len(tokens))},
    ),
    html.Br(),
    dcc.Graph(
        id = "mapbox",
        #animate=True,
        style = {"height": 750}
    )])

@app.callback(
    dash.dependencies.Output('mapbox', 'figure'),
    [dash.dependencies.Input('slider', 'value')])
def update_graph(value):
    print(value)
    return {
        'data': data[value]['data'],
        'layout': data[value]['layout']
    }

app.run_server(debug=False, port=8012)

 * Running on http://127.0.0.1:8012/ (Press CTRL+C to quit)
127.0.0.1 - - [26/May/2018 15:56:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2018 15:56:40] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2018 15:56:40] "GET /_dash-dependencies HTTP/1.1" 200 -


0


127.0.0.1 - - [26/May/2018 15:56:42] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2018 15:56:43] "POST /_dash-update-component HTTP/1.1" 200 -


1


127.0.0.1 - - [26/May/2018 15:57:20] "POST /_dash-update-component HTTP/1.1" 200 -


...

127.0.0.1 - - [26/May/2018 15:58:31] "POST /_dash-update-component HTTP/1.1" 200 -


3


127.0.0.1 - - [26/May/2018 15:59:03] "POST /_dash-update-component HTTP/1.1" 200 -


4


127.0.0.1 - - [26/May/2018 15:59:18] "POST /_dash-update-component HTTP/1.1" 200 -


In [11]:
#
